# Create Dataset

Code to create the dataset (requires parsee-core install)

You have to set the Parsee Cloud API key and the output directory here

In [4]:
parsee_api_key = "API_KEY"
output_directory = "PATH_TO_OUTPUT_DIR"

We are setting a token limit of 8k tokens to all prompts, this should not affect the prompts for this dataset, but if you are using larger files, increasing this is probably better (depends on the exact use case).

In [5]:
token_limit_prompts = 8000

For the dataset creation, we will load the raw PDF files (included in this repo):

In [6]:
data_dir = "../raw_data"

code to create dataset:

In [ ]:
import os

from parsee.cloud.api import ParseeCloud
from parsee.datasets.writers.disk_writer import CsvDiskWriter
from parsee.datasets.main import create_dataset_rows
from parsee.converters.langchain import langchain_loader_to_sdf
from parsee.utils.enums import DocumentType
from parsee.utils.helper import get_source_identifier

from langchain.document_loaders.pdf import PyPDFLoader

cloud = ParseeCloud(parsee_api_key)
template = cloud.get_template("65f95cc2e34036446ee85a00") # this is the revenues template

documents = [x for x in os.listdir(data_dir) if x.endswith(".pdf")]
writer = CsvDiskWriter(output_directory)


for doc_path in documents:
    full_path = os.path.join(data_dir, doc_path)
    parsee_document = langchain_loader_to_sdf(PyPDFLoader(full_path), DocumentType.PDF, get_source_identifier(full_path))
    # get correct values from parsee cloud
    correct_values = cloud.get_output(parsee_document.source_identifier, template.id)
    dataset_rows = create_dataset_rows(template, parsee_document, [x.to_assigned_answer() for x in correct_values], max_tokens_prompt=token_limit_prompts)

    # write the rows at the target destination as CSV
    writer.write_rows(dataset_rows, "questions_revenues_langchain")